# 2_DataWrangling: Price Predictor

In [1]:
# Import for data wrangling
import pandas as pd

## Loading Data

1. Loading files
2. Concatenating into a single file
3. Basic data exploration

In [22]:
# Read each file separately
df1 = pd.read_csv('../../data/price_predictor/raw/Sample_Product_Data_Batch_1.csv')
df2 = pd.read_csv('../../data/price_predictor/raw/Sample_Product_Data_Batch_2.csv')
df3 = pd.read_csv('../../data/price_predictor/raw/Sample_Product_Data_Batch_3.csv')
df4 = pd.read_csv('../../data/price_predictor/raw/Sample_Product_Data_Batch_4.csv')

# Concatenate dataframes
supply_ch_df_raw = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Save the concatenated raw dataframe
supply_ch_df_raw.to_csv('../../data/price_predictor/raw/Sample_Product_Data_Concat.csv', index=False)

---

#### Dataset initial inspection

In [23]:
# Data glance
supply_ch_df_raw.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,cosmetics,062000005,4.35,98.0,216.0,$5513.64,NaN,28.0,5.0,16.0,...,Kolkata,28.0,406.0,9.0,$98.49,Fail,3.27,Rail,A route,329.95
1,cosmetics,062000004,31.84,11.0,407.0,$1638.02,Unknown,41.0,7.0,59.0,...,NaN,6.0,599.0,15.0,$67.15,Fail,0.52,Rail,c,NaN
2,haircare,062000002,78.27,10.0,399.0,NaN,Male,66.0,26.0,81.0,...,Bangalore,12.0,721.0,28.0,$49.86,NaN,1.27,Road,a,996.28
3,haircare,062000003,NaN,15.0,923.0,$9417.67,Non-binary,25.0,26.0,NaN,...,Bangalore,11.0,NaN,12.0,$54.2,Fail,NaN,Sea,C route,817.38
4,cosmetics,062000007,22.79,17.0,715.0,$8656.12,Non-binary,58.0,24.0,79.0,...,Kolkata,16.0,168.0,21.0,$71.42,Pending,3.35,Rail,b,407.65


In [24]:
# Data info
supply_ch_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             9170 non-null   object 
 1   SKU                      10000 non-null  object 
 2   Price                    9470 non-null   float64
 3   Availability             9230 non-null   float64
 4   Number of products sold  9150 non-null   float64
 5   Revenue generated        9250 non-null   object 
 6   Customer demographics    9330 non-null   object 
 7   Stock levels             9450 non-null   float64
 8   Lead times               9310 non-null   float64
 9   Order quantities         9430 non-null   float64
 10  Shipping times           9110 non-null   float64
 11  Shipping carriers        9260 non-null   object 
 12  Shipping costs           9160 non-null   object 
 13  Supplier name            9080 non-null   object 
 14  Location               

---

#### Dataset unique values

In [25]:
# Number of unique values per column
supply_ch_df_raw.nunique()

Product type                   3
SKU                        10000
Price                       6106
Availability                  99
Number of products sold      999
Revenue generated           9205
Customer demographics          4
Stock levels                  99
Lead times                    29
Order quantities              99
Shipping times                 9
Shipping carriers              3
Shipping costs               901
Supplier name                  5
Location                       5
Lead time                     29
Production volumes           900
Manufacturing lead time       29
Manufacturing costs         5798
Inspection results             3
Defect rates                 501
Transportation modes           4
Routes                        15
Costs                       8920
dtype: int64

In [26]:
# List of columns to get unique values for
columns_to_check = [
    'Product type', 'Availability', 'Customer demographics', 'Stock levels', 'Lead times', 
    'Order quantities', 'Shipping times', 'Shipping carriers', 'Supplier name', 'Location', 
    'Lead time', 'Manufacturing lead time', 'Inspection results', 'Transportation modes', 'Routes'
]

# Iterate over the columns and print unique values
for column in columns_to_check:
    unique_values = supply_ch_df_raw[column].unique()
    print(f"Unique values in '{column}'\n: {unique_values}\n")

Unique values in 'Product type'
: ['cosmetics' 'haircare' 'skincare' nan]

Unique values in 'Availability'
: [98. 11. 10. 15. 17. 40. 18. 55. nan 25. 78. 89. 22. 30. 84. 85. 44. 32.
 51. 46. 21. 99. 94. 39. 12. 47. 13.  8.  7. 23. 96. 75. 86. 92. 29.  2.
 79. 52. 27. 50. 95. 88. 77. 43. 37. 61. 97. 56.  3. 63. 19. 58. 16. 72.
  1.  5. 65.  4. 64. 91. 41.  9. 36. 90. 54. 81. 14. 34. 53. 20. 70. 83.
 24. 74. 38. 57. 45. 71. 93. 82. 31. 60. 49. 76. 48. 33. 69. 26. 35. 80.
 73. 68. 42. 87. 28. 66. 59.  6. 62. 67.]

Unique values in 'Customer demographics'
: [nan 'Unknown' 'Male' 'Non-binary' 'Female']

Unique values in 'Stock levels'
: [28. 41. 66. 25. 58.  6. 75. 38. 45. 14. 40.  3. 99. 97. nan 19. 18. 64.
 88. 35. 62.  1. 49. 98. 89. 16. 93. 23. 83. 54. 73. 42. 34. 80. 55. 21.
 70. 96. 48. 39. 31. 10. 30. 60. 15.  4. 63.  9. 26. 81. 52. 69.  5. 20.
 79.  2. 13. 74. 61. 95.  8. 67. 94. 11. 36. 12. 37. 71. 56. 85. 29. 78.
 90. 92. 82. 84. 86. 59. 68.  7. 51. 72. 17. 77. 27. 24. 87. 46. 91.

This unique values validation gives us an idea that there may not be any repeated columns since the SKUs (shipments IDs) are unique in all records.

---

### Dataset missing missing values

In [27]:
# Checking missing value count in each column
print("\nMissing values in each column:")
missing_values = supply_ch_df_raw.isnull().sum()
display(missing_values)


Missing values in each column:


Product type               830
SKU                          0
Price                      530
Availability               770
Number of products sold    850
Revenue generated          750
Customer demographics      670
Stock levels               550
Lead times                 690
Order quantities           570
Shipping times             890
Shipping carriers          740
Shipping costs             840
Supplier name              920
Location                   720
Lead time                  950
Production volumes         700
Manufacturing lead time    570
Manufacturing costs        680
Inspection results         630
Defect rates               590
Transportation modes       920
Routes                     840
Costs                      590
dtype: int64

In [28]:
# Calculate the percentage of null values for each column
null_percentage_per_column = supply_ch_df_raw.isnull().mean() * 100
print("Percentage of null values per column:\n", null_percentage_per_column)

# Calculate the overall percentage of null values in the entire dataset
total_null_values = supply_ch_df_raw.isnull().sum().sum()
total_values = supply_ch_df_raw.size
overall_null_percentage = (total_null_values / total_values) * 100
print(f'Overall percentage of null values: {overall_null_percentage}%')

# Check for empty rows
rows_with_null = sum(supply_ch_df_raw.isnull().all(axis=1))
print(f'Number of empty rows: {rows_with_null}')


Percentage of null values per column:
 Product type               8.3
SKU                        0.0
Price                      5.3
Availability               7.7
Number of products sold    8.5
Revenue generated          7.5
Customer demographics      6.7
Stock levels               5.5
Lead times                 6.9
Order quantities           5.7
Shipping times             8.9
Shipping carriers          7.4
Shipping costs             8.4
Supplier name              9.2
Location                   7.2
Lead time                  9.5
Production volumes         7.0
Manufacturing lead time    5.7
Manufacturing costs        6.8
Inspection results         6.3
Defect rates               5.9
Transportation modes       9.2
Routes                     8.4
Costs                      5.9
dtype: float64
Overall percentage of null values: 6.995833333333333%
Number of empty rows: 0


---

### Handling Missing values

**DW-1: Dropping null values**

Given that the percentage of null values in the dataset is less than 7% overall, we will drop those rows with null values for the data cleaning

In [29]:
# Dropping rows with null values
supply_ch_df_cleaned = supply_ch_df_raw.dropna()

In [30]:
supply_ch_df_cleaned.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
4,cosmetics,062000007,22.79,17.0,715.0,$8656.12,Non-binary,58.0,24.0,79.0,...,Kolkata,16.0,168.0,21.0,$71.42,Pending,3.35,Rail,b,407.65
11,cosmetics,1VBA0260GE,53.97,78.0,949.0,$4516.14,Unknown,75.0,15.0,90.0,...,Delhi,3.0,617.0,28.0,$80.63,Pass,4.66,Rail,a,424.80
17,skincare,1VBF0810GE,80.18,44.0,687.0,$6449.54,Male,97.0,20.0,10.0,...,Delhi,13.0,306.0,17.0,$65.34,Pending,0.61,Air,b,603.41
20,skincare,1VMG0080GE,62.87,46.0,903.0,$1755.28,Female,18.0,21.0,98.0,...,Delhi,20.0,486.0,29.0,$29.09,Fail,3.09,Rail,C route,331.07
22,skincare,1VMG0140GE,83.82,99.0,163.0,$7097.87,Non-binary,88.0,18.0,50.0,...,Chennai,23.0,941.0,17.0,$97.18,Pending,2.58,Road,route_B,392.25


---

### Handling Data Types

Given the information obtain from the initial basic inspection in the dataset, almost all columns seem to be ad-goc with their data types. However, there are a couple of columns that need further adjstments.

**DW-2: Special characters in currency columns**

Columns like `Revenue generated`, `Shipping costs` and `Manufacturing costs` have dollar signs in their numerical values. Therefore, they need transformation for proper processing.

In [31]:
# Verify data
supply_ch_df_cleaned[['Revenue generated', 'Shipping costs', 'Manufacturing costs']].head()

,Revenue generated,Shipping costs,Manufacturing costs
4,$8656.12,$4.7,$71.42
11,$4516.14,$8.68,$80.63
17,$6449.54,$6.37,$65.34
20,$1755.28,$2.1,$29.09
22,$7097.87,$6.27,$97.18


In [32]:
# Removing dollar sign and passing to numeric type
for column in supply_ch_df_cleaned[['Revenue generated', 'Shipping costs', 'Manufacturing costs']]:
    supply_ch_df_cleaned.loc[:, column] = supply_ch_df_cleaned.loc[:, column].str.replace('[$,]', '', regex=True).astype(float)

---

### Handling Consistency in Records

In [33]:
# Displaying different representations of route instance
supply_ch_df_cleaned['Routes'].unique()

array(['b', 'a', 'C route', 'route_B', 'B route', 'c', 'routeC', 'routeA',
       'routeB', 'B', 'A route', 'route_A', 'C', 'A', 'route_C'],
      dtype=object)

**DW-3: Different representations of same instances**

If we have a closer look to the routes column, we can see that there are multiple representatios of the same route. Therefore, we need to make that homogeneous for the data processing.

In [34]:
# Mapping dictionary
route_mapping = {
    'a': 'Route_A', 'A': 'Route_A', 'A route': 'Route_A', 'routeA': 'Route_A', 'route_A': 'Route_A',
    'b': 'Route_B', 'B': 'Route_B', 'B route': 'Route_B', 'routeB': 'Route_B', 'route_B': 'Route_B',
    'c': 'Route_C', 'C': 'Route_C', 'C route': 'Route_C', 'routeC': 'Route_C', 'route_C': 'Route_C'
}

# Apply the mapping to the 'Routes' column
supply_ch_df_cleaned.loc[:, 'Routes'] = supply_ch_df_cleaned.loc[:, 'Routes'].replace(route_mapping)

---

### Handling Column Names

**DW-4: Renaming Columns**

Now that we have the final selected columns in out dataset, we will rename thos so we can avoid pecial characters (if any) and any spaces on them.

In [35]:
supply_ch_df_cleaned.columns

Index(['Product type', 'SKU', 'Price', 'Availability',
       'Number of products sold', 'Revenue generated', 'Customer demographics',
       'Stock levels', 'Lead times', 'Order quantities', 'Shipping times',
       'Shipping carriers', 'Shipping costs', 'Supplier name', 'Location',
       'Lead time', 'Production volumes', 'Manufacturing lead time',
       'Manufacturing costs', 'Inspection results', 'Defect rates',
       'Transportation modes', 'Routes', 'Costs'],
      dtype='object')

In [36]:
# Renaming columns
supply_ch_df_cleaned.columns = supply_ch_df_cleaned.columns.str.replace(' ', '_').str.replace(r'\W', '', regex=True).str.lower()

---

### DW - Data Cleaning Validations

In [37]:
supply_ch_df_cleaned.head()

,product_type,sku,price,availability,number_of_products_sold,revenue_generated,customer_demographics,stock_levels,lead_times,order_quantities,...,location,lead_time,production_volumes,manufacturing_lead_time,manufacturing_costs,inspection_results,defect_rates,transportation_modes,routes,costs
4,cosmetics,062000007,22.79,17.0,715.0,8656.12,Non-binary,58.0,24.0,79.0,...,Kolkata,16.0,168.0,21.0,71.42,Pending,3.35,Rail,Route_B,407.65
11,cosmetics,1VBA0260GE,53.97,78.0,949.0,4516.14,Unknown,75.0,15.0,90.0,...,Delhi,3.0,617.0,28.0,80.63,Pass,4.66,Rail,Route_A,424.80
17,skincare,1VBF0810GE,80.18,44.0,687.0,6449.54,Male,97.0,20.0,10.0,...,Delhi,13.0,306.0,17.0,65.34,Pending,0.61,Air,Route_B,603.41
20,skincare,1VMG0080GE,62.87,46.0,903.0,1755.28,Female,18.0,21.0,98.0,...,Delhi,20.0,486.0,29.0,29.09,Fail,3.09,Rail,Route_C,331.07
22,skincare,1VMG0140GE,83.82,99.0,163.0,7097.87,Non-binary,88.0,18.0,50.0,...,Chennai,23.0,941.0,17.0,97.18,Pending,2.58,Road,Route_B,392.25


**DW-1**

In [38]:
# Calculate the percentage of null values for each column
null_percentage_per_column = supply_ch_df_cleaned.isnull().mean() * 100
print("Percentage of null values per column:\n", null_percentage_per_column)

# Calculate the overall percentage of null values in the entire dataset
total_null_values = supply_ch_df_cleaned.isnull().sum().sum()
total_values = supply_ch_df_cleaned.size
overall_null_percentage = (total_null_values / total_values) * 100
print(f'Overall percentage of null values: {overall_null_percentage}%')

# Check for empty rows
rows_with_null = sum(supply_ch_df_cleaned.isnull().all(axis=1))
print(f'Number of empty rows: {rows_with_null}')

Percentage of null values per column:
 product_type               0.0
sku                        0.0
price                      0.0
availability               0.0
number_of_products_sold    0.0
revenue_generated          0.0
customer_demographics      0.0
stock_levels               0.0
lead_times                 0.0
order_quantities           0.0
shipping_times             0.0
shipping_carriers          0.0
shipping_costs             0.0
supplier_name              0.0
location                   0.0
lead_time                  0.0
production_volumes         0.0
manufacturing_lead_time    0.0
manufacturing_costs        0.0
inspection_results         0.0
defect_rates               0.0
transportation_modes       0.0
routes                     0.0
costs                      0.0
dtype: float64
Overall percentage of null values: 0.0%
Number of empty rows: 0


**DW-2**

In [41]:
supply_ch_df_cleaned[['revenue_generated', 'shipping_costs', 'manufacturing_costs']].head()

,revenue_generated,shipping_costs,manufacturing_costs
4,8656.12,4.7,71.42
11,4516.14,8.68,80.63
17,6449.54,6.37,65.34
20,1755.28,2.1,29.09
22,7097.87,6.27,97.18


**DW-3**

In [42]:
# Displaying different representations of route instance
supply_ch_df_cleaned['routes'].unique()

array(['Route_B', 'Route_A', 'Route_C'], dtype=object)

---

### Saving cleaned dataset for EDA ingestion

In [43]:
# Save the cleaned dataframe
supply_ch_df_cleaned.to_csv('../../data/price_predictor/staged/Sample_Product_Data_Cleaned.csv', index=False)